# Can we identify clusters in our movies

In [7]:
# import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [8]:
df = pd.read_pickle('Pickles/cos_sim_df.pkl')
# df = pd.read_pickle('Pickles/pca_100_df.pkl')


df.shape


(1343, 1344)

# Dynamic Hierarchical Clustering
- we want our number of clusters to be dynamic
- we can do this by setting the distance threshold, such that only the movies above this threshold are valid for the cluster
- .1 would result in clusters being formed with >.9 cosine affinity

In [9]:
100/9904

0.010096930533117932

In [10]:
df.head(2)

movieId,1,2,3,5,6,7,10,11,14,16,...,134130,134853,139385,142488,148626,152081,164179,166528,168252,movieId
0,1.00000,-0.04729,-0.007196,-0.020456,0.003208,-0.052876,0.014008,-0.004965,-0.007231,0.164520,...,-0.002414,-0.001942,0.033379,0.075848,-0.004270,-0.037548,0.007200,-0.046395,-0.051469,1
1,-0.04729,1.00000,-0.101892,-0.006796,0.086783,-0.089670,-0.006500,0.062502,-0.031550,-0.113855,...,-0.010612,0.102774,0.094325,0.062686,0.035455,0.170830,-0.008577,-0.045373,-0.043255,2


# Convert cosine similarity to distanc emetric

In [11]:
X = df.drop(columns=['movieId'], axis=1)

X = X.subtract(1)

X = X.abs()

In [12]:
X.head(2)

movieId,1,2,3,5,6,7,10,11,14,16,...,128360,134130,134853,139385,142488,148626,152081,164179,166528,168252
0,2.220446e-16,1.04729,1.007196,1.020456,0.996792,1.052876,0.985992,1.004965,1.007231,0.835480,...,1.062118,1.002414,1.001942,0.966621,0.924152,1.004270,1.037548,0.992800,1.046395,1.051469
1,1.047290e+00,0.00000,1.101892,1.006796,0.913217,1.089670,1.006500,0.937498,1.031550,1.113855,...,1.015452,1.010612,0.897226,0.905675,0.937314,0.964545,0.829170,1.008577,1.045373,1.043255


In [24]:
from sklearn.cluster import AgglomerativeClustering

# X = df.drop(columns=['Title'], axis=1)

# use distance threshold and n clusters is dynamic
clustering = AgglomerativeClustering(n_clusters=None, 
                                     affinity='precomputed', 
                                     linkage='complete', 
                                     compute_full_tree=True,
                                     distance_threshold=.5).fit(X)



cluster_labels = clustering.labels_.tolist() # store labels as a list
cluster_titles = df.movieId.tolist() # store titles as a list

# dict with our cluster number as key
cluster_dict = {}
for i,v in enumerate(cluster_labels):
    if v in cluster_dict:
        cluster_dict[v].append(cluster_titles[i])
    else:
        cluster_dict[v] = [cluster_titles[i]]
print("Number of clusters: {n}".format(n=len(cluster_dict)))

# dict with our movie title as key
title_dict = {}
for i,v in enumerate(cluster_titles):
    title_dict[v] = cluster_labels[i]


Number of clusters: 585


# Cluster Dataframe

- we need a dataframe with each cluster as a sample.

In [25]:
df_data_list = []
for k,v in cluster_dict.items():
    df_data_list.append({'cluster': k,
                         'movies': v})
    
cluster_df = pd.DataFrame(df_data_list)

cluster_df['cluster_size'] = [len(i) for i in cluster_df.movies]
cluster_df = cluster_df.sort_values(by='cluster_size', ascending=False)
cluster_df.head()

,cluster,movies,cluster_size
53,96,"[168, 1275, 2947, 2948, 2949, 2991, 2993, 3635...",11
40,154,"[110, 1225, 1299, 1416, 1960, 2336, 30749, 314...",11
57,186,"[175, 193, 926, 1103, 1124, 1172, 3083, 3160, ...",11
72,156,"[232, 308, 902, 909, 910, 916, 1235, 2324, 529...",10
68,41,"[215, 222, 265, 306, 4014, 5015, 8533, 8638, 6...",9


In [32]:
cluster_df.loc[cluster_df['cluster_size']==1].shape

(220, 3)

In [28]:
cluster_df.cluster_size.mean()

2.2957264957264956

# Pickle the cluster dataframe 

In [27]:
cluster_df.to_pickle('Pickles/clusterPrecomputed_df.pkl')